## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = [path.curdir, "Sorting-S"]
project_path

['.', 'Sorting-S']

In [6]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:12925
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x000006c4.
[DEBUG] Memory segment base written at 0x000006c4. Size = 0x00000400.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_sorting"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment Sorting",
}

Stop criteria to pass to the logger:

In [18]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [19]:
def stop_function(pin_values):
    return all(pin_values)

In [20]:
repetitions = 10

Perform the measurement.

In [21]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges = []
times = []

with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2,stop_fn)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) > 1 or len(time) > 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        # dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {charge[0]*1e3:.06} mC, time: {time[0]:.06} s")
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))
c:\users\erikw_000\documents\github\atmel-saml11\python\pydgilib\pydgilib_extra\dgilib_calculations.py:209: UserWarning: Corrected a start_index of 0 in power_and_time_per_pulse.
  "power_and_time_per_pulse.")


IndexError: list index out of range

In [12]:
import json

config = {}
config["name"] = "Sorting 1024"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Sorting"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": [], 
                                        "loop": ["Sorting"],
                                        "exit": []}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
sum(charges)/repetitions

In [ ]:
print("%.0100f" % (sum(charges)/repetitions))